In [2]:
import pandas as pd
import anthropic
from dotenv import load_dotenv
import os
import json
from openpyxl import load_workbook
from openai import OpenAI


# load the API key from the .env file
load_dotenv()
api_key = os.getenv("ANTHROPIC_API_KEY")

api_key_openai = os.getenv("OPENAI_API_KEY")


# initialize the client
client = anthropic.Anthropic(api_key=api_key)
# clientOpenAI = OpenAI(api_key=api_key_openai)


# Enter the Error IDs here to rerun the API call for these and replace the existing rows in the output Excel file
error_ids = []
ids = []  # Enter the IDs here

# Global variables
counter = 0  # To keep track of processed rows

# Load the input CSV file and the output Excel file
input_path = 'Dataextraktion_Piloting_Literatur.csv'
output_path = 'Dataextraktion_Piloting.xlsx'  # Specify the output Excel file
df = pd.read_csv(input_path, sep=';', encoding='utf-8-sig')  
df_output = pd.read_excel(output_path)  


# Filter the input DataFrame to only include the rows with error IDs
if len(ids) > 0: 
    df = df[df['ID'].isin(ids)]

if len(error_ids) > 0:
    df = df[df['ID'].isin(error_ids)]


# Start the counter from the max ID in the output file to continue from where it left off
# max_id = df_output['ID'].max()
# if pd.isna(max_id):
#     print("No valid IDs in the file.")
# else:
#     print(f"Max ID: {max_id}")
#     df = df[df['ID'] > max_id]


def append_to_excel(row, output_path):
    global counter
    row_df = pd.DataFrame([row])

    # Load existing workbook and sheet
    workbook = load_workbook(output_path)
    sheet = workbook.active

    # Check if ID exists in the sheet
    existing_ids = [cell.value for cell in sheet['A']]  # Assuming 'A' is the column where IDs are stored
    print(f"Existing IDs: {existing_ids}")

    # If ID exists, replace the row
    # if row['ID'] in existing_ids and row['ID'] in error_ids:
    #     # Find the row with the matching ID and replace it
    #     row_index = existing_ids.index(row['ID']) + 1  # Excel is 1-indexed
    #     for col_num, value in enumerate(row_df.iloc[0], start=1):
    #         sheet.cell(row=row_index, column=col_num, value=value)
    #         row["Error"] = "Error"
    #     print(f"Replaced ID: {row['ID']}")
    # else:
        # Otherwise, append the new row
    startrow = sheet.max_row + 1
    for col_num, value in enumerate(row_df.iloc[0], start=1):
        sheet.cell(row=startrow, column=col_num, value=value)
    print(f"Appended ID: {row['ID']}")

    # Save the workbook
    workbook.save(output_path)
    counter += 1
    print(f"Counter: {counter}")


def extractInformation(row):

    try:
        title = row['Title']
        abstract = row['Abstract']

        systemprompt = "You are a reviewer conducting a systematic literature review. Extract information from the title and abstract provided to answer the review questions in the strict context of the content provided."
        
        prompt = f"""
            Extract the information from title and abstract following the instructions below:

            1. Review the list of questions provided in <review_questions>. Understand what specific information each question seeks.
            1. Carefully read the text provided in <title> and <abstract>. Look for information directly related to the review questions.
            2. For each review question try to answer the question based on the information found in the title and abstract. Just answer if you are sure that the information and match the context of the question.
                - Locate Information: Find relevant information in the title or abstract that answers the question.
                - Match Context: Ensure the answer directly aligns with the review question's context. Be vigilant for misleading or unrelated information.
                - Provide Explanation: Include a brief explanation that summarizes why the information supports the answer and quote the original text from the title or abstract in the ***explanation*** field.
                - If the title and abstract do not provide sufficient information for a question: Leave the answer field empty. In the explanation field, briefly state why the question could not be answered (e.g., "No relevant information found in the title or abstract")
            3. Set CountAnswered to the number of questions for which answers were provided.
            4. Construct a JSON object using the structure in <desired_output> and return it. Note: Is just an example for the structure of the output!
            
            <title>
            {title}
            </title>

            <abstract>
            {abstract}
            </abstract>
            
            <review_questions>
            - **RQ1**: Which machine learning methods are used for deepfake detection? (e.g. Logistic Regression, Support Vector Machines (SVMs), Convolutional Neural Networks (CNNs), Generative Adversarial Networks (GANs) etc)
            - **RQ2**: Which machine learning methods are recommended (and thus particularly suitable)? (e.g. Logistic Regression, Support Vector Machines (SVMs), Convolutional Neural Networks (CNNs), Generative Adversarial Networks (GANs) etc)
            - **RQ3**: What challenges exist in detecting deepfakes using machine learning approaches? (e.g. outdated or limited datasets, Generalization issues across datasets, Rapid evolution of deepfake techniques, High computational cost of detection models, Difficulty in detecting low-quality or compressed media etc)
            - **RQ4**: What are the use cases for deepfake detection? (e.g., COVID-19 masks, medicine, media, politics etc)     
            </review_questions> 
            
        <desired_output>
            {{
                "RQ1": {{
                    "answer": "Some Information/Answer for RQ1"
                    "explanation": "brief explanation and original text here"
                }},
                "RQ2": {{
                    "answer": "",
                    "explanation": "brief explanation here why it was not possible to answer the question"
                }},
                "RQ3": {{
                    "answer": "",
                    "explanation": "brief explanation here why it was not possible to answer the question"
                }},
                "RQ4": {{
                    "answer": "Some Information/Answer for RQ4"
                    "explanation": "brief explanation here and original text here"
                }}, 
                "CountAnswered": 2
                
            }}
            </desired_output>

            Just return the json object according to this structure. With no beginning ```json or so. Just the object
            """
        # API-Aufruf Claude
        response = client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=2048,
            system=systemprompt,
            temperature=0,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )

        # API-Aufruf ChatGPT#
        # response = clientOpenAI.chat.completions.create(
        # model="gpt-4o",
        # temperature=0,
        #  messages=[
        #         {"role": "system", "content": systemprompt},
        #         {"role": "user", "content": prompt}
        #     ])
        # completeRepsonse = response

        # get the content of the response
        
        # Claude - Response
        content = response.content[0].text

        # Open AI - Response
        # content = response.choices[0].message.content

        data = json.loads(content)
        # extract the relevant information
    
        # add the relevant information to the row
        row["Auto CountAnswered"] = data["CountAnswered"]
        row["Matched"] = row["Manual CountAnswered"] == row["Auto CountAnswered"]
        row["Match Var"] = ""

        row["RQ1"] = data["RQ1"].get("answer", None)
        row["RQ2"] = data["RQ2"].get("answer", None)
        row["RQ3"] = data["RQ3"].get("answer", None)
        row["RQ4"] = data["RQ4"].get("answer", None)

        row["RQ1_Explaination"] = data["RQ1"].get("explanation", None)
        row["RQ2_Explaination"] = data["RQ2"].get("explanation", None)
        row["RQ3_Explaination"] = data["RQ3"].get("explanation", None)
        row["RQ4_Explaination"] = data["RQ4"].get("explanation", None)


        # Claude API - Usage
        row["input_tokens"] = response.usage.input_tokens
        row["output_tokens"] = response.usage.output_tokens

        # OpenAI - Usage
        # row["input_tokens"] = response.usage.completion_tokens
        # row["output_tokens"] = response.usage.completion_tokens

    except Exception as e:
        # falls ein Fehler auftritt, füge die ID zur error_ids-Liste hinzu
        print(f"Error bei ID: {row['ID']} : {e}")
        # add the ID to the error_ids list
        error_ids.append(row['ID'])
        row["RQ1"] = None
        row["RQ2"] = None
        row["RQ3"] = None
        row["RQ4"] = None
        row["CountAnswered"] = None
        row["input_tokens"] = None
        row["output_tokens"] = None


    append_to_excel(row, output_path)
    return row

# apply the function to the DataFrame
df = df.apply(extractInformation, axis=1)
print(f"Error IDs: {error_ids}")
print(f"Counter: {counter}")
print("Done!")



Processed ID: 9
Processed ID: 24
Processed ID: 29
Processed ID: 67
Processed ID: 103
Processed ID: 146
Processed ID: 149
Processed ID: 254
Processed ID: 259
Processed ID: 277
Processed ID: 292
Processed ID: 327
Processed ID: 344
Processed ID: 424
Processed ID: 488
Processed ID: 584
Processed ID: 814
Processed ID: 820
Processed ID: 898
Processed ID: 917
Error IDs: []
Counter: 0
Done!
